In [4]:
import requests
import tweepy
import json
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm
import requests



In [ ]:
at = ''
ats = ''
bt = ''

api_key = ''
api_secret_key = ''
access_token = ''
access_token_secret = ''

Creating time period for gathering tweets, after collecting counts for all tweets within segments of 30 days

In [ ]:
dates = [str(i)[:-9] + 'T00:00:00Z' for i in pd.date_range(start='11/20/2022', periods=150)] # initialising 150 days period 

MY_BEARER_TOKEN = bt
client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

master = []

for i in range(0, len(dates)//30 + 2):
    data2 = client.get_all_tweets_count(query='balenciaga',
                         start_time=dates[i * 30],
                         end_time=dates[(i+1) * 30]
                         )
    
    master.append(pd.DataFrame(data2.data))

Here one should watch for the "TooManyRequests: 429" error which occurs when the client has exceeded the rate limit of the API service that we're using. API providers often impose rate limits to prevent abuse and ensure fair usage of their services. If you exceed the rate limit, the API will return a "TooManyRequests" error. To resolve this error, one should wait for some time (usually a few minutes) before retrying the request. The exact duration of the wait time depends on the specific API service and rate limit policy. The most common request limit interval is fifteen minutes. Our endpoint has a rate limit of 300 requests/15 minutes, then up to 300 requests over any 15-minute interval are allowed. Our code will handle rate limiting properly by retrying the request after waiting for 900 seconds (15 minutes) using time.sleep() when it reaches the maximum requests possible.

In [ ]:

pairs_arr = pd.DataFrame({'start':dates[:-1], 'end':dates[1:]})


MY_BEARER_TOKEN = bt
client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

counter = 0

dfs = []

for search_query in tqdm(['balenciaga']):
    for i in range(pairs_arr.shape[0]):

        try:
            '''
            fetches tweet counts for the specified search query ('balenciaga')
            within the specified time range (defined by start_time and end_time) with a granularity of 'minute'.
            '''
            tweets = client.get_all_tweets_count(query=search_query,
                                                 start_time=pairs_arr['start'][i],
                                                 end_time=pairs_arr['end'][i],
                                                 granularity='minute'
                                                 )
            df = pd.DataFrame(tweets.data)
            print(search_query, df['tweet_count'].sum())

            df['search_query'] = search_query

            counter += df['tweet_count'].sum()

            dfs.append(df)

        except Exception as e:
            print(e)
            print('Waiting mode')
            time.sleep(900)

            '''
            after sleeping we are trying to fetch tweets counts again
            '''

            tweets = client.get_all_tweets_count(query=search_query,
                                                 start_time=pair[0],
                                                 end_time=pair[1],
                                                 granularity='minute'
                                                 )
            df = pd.DataFrame(tweets.data)
            print(search_query, df['tweet_count'].sum())

            df['search_query'] = search_query

            counter += df['tweet_count'].sum()

            dfs.append(df)

dfs = pd.concat(dfs, axis=0)
dfs.to_pickle('time_ranges1.pkl')

The purpose of this loop is to assign each row to a specific time period based on the number of tweets associated with the tag within that time period. In general, it is grouping together contiguous time periods. There is a 500-requests limit, so first you need to collect the time ranges and then aggregate them to have the most optimal time intervals 500 tweets per request limit. 

In [ ]:
### categorizes tweet counts into different time periods
### counter2  is used to track the cumulative tweet count over multiple time periods
### counter for  the tweet count within a single time period.

tags = ['balenciaga']

df_all = []

for tag in tags:
    df_local = dfs.copy().reset_index(drop=True)

    counter = 0
    counter2 = 0

    for i in tqdm(range(1, df_local.shape[0])):
        counter2 += df_local['tweet_count'][i]
        
        if df_local['tweet_count'][i] >= 450:
            if df_local['tweet_count'][i-1] < 450:
                df_local.at[i-1, 'timeperiod'] = 1
            df_local.at[i, 'timeperiod'] = 1
            counter = 0
            counter2 = 0
        elif counter2 >= 450 and counter < 450 and counter > 0:
            df_local.at[i-1, 'timeperiod'] = 1
            counter = df_local['tweet_count'][i]
            counter2 = df_local['tweet_count'][i]
        elif counter2 < 450 and counter < 450:
            counter += df_local.tweet_count[i]
        elif counter == 0 and counter2 > 450:
            df_local.at[i, 'timeperiod'] = 2
            counter = 0
            counter2 = 0
        
    df_all.append(df_local)
    
df_all = pd.concat(df_all, axis = 0)
df_all.reset_index(inplace=True, drop=True)

df_all2 = []


df_local = df_all_[df_all_['search_query'] == tag].reset_index(drop=True)
ends = df_local['end'].values.tolist()[:-1]
starts = [df_local['start'][0]] + ends
df_local['start'] = starts
df_all2.append(df_local)
    
df_all2 = pd.concat(df_all2, axis=0)
df_all2.to_pickle('intervals.pkl')

In [122]:
df_all2

,end,start,tweet_count,search_query,timeperiod
0,2022-11-20T00:01:00.000Z,2022-11-20T00:00:00.000Z,4,balenciaga,1.0
1,2022-11-20T02:09:00.000Z,2022-11-20T00:01:00.000Z,5,balenciaga,1.0
2,2022-11-20T03:37:00.000Z,2022-11-20T02:09:00.000Z,5,balenciaga,1.0
3,2022-11-20T04:39:00.000Z,2022-11-20T03:37:00.000Z,4,balenciaga,1.0
4,2022-11-20T06:28:00.000Z,2022-11-20T04:39:00.000Z,5,balenciaga,1.0
...,...,...,...,...,...
6257,2023-04-11T13:40:00.000Z,2023-04-11T10:55:00.000Z,4,balenciaga,1.0
6258,2023-04-11T16:00:00.000Z,2023-04-11T13:40:00.000Z,4,balenciaga,1.0
6259,2023-04-11T18:51:00.000Z,2023-04-11T16:00:00.000Z,5,balenciaga,1.0
6260,2023-04-11T22:11:00.000Z,2023-04-11T18:51:00.000Z,2,balenciaga,1.0


In [129]:
ch = pd.read_pickle('intervals.pkl')
ch.reset_index(inplace=True, drop=True)
ch

,end,start,tweet_count,search_query,timeperiod
0,2022-11-20T00:01:00.000Z,2022-11-20T00:00:00.000Z,4,balenciaga,1.0
1,2022-11-20T02:09:00.000Z,2022-11-20T00:01:00.000Z,5,balenciaga,1.0
2,2022-11-20T03:37:00.000Z,2022-11-20T02:09:00.000Z,5,balenciaga,1.0
3,2022-11-20T04:39:00.000Z,2022-11-20T03:37:00.000Z,4,balenciaga,1.0
4,2022-11-20T06:28:00.000Z,2022-11-20T04:39:00.000Z,5,balenciaga,1.0
...,...,...,...,...,...
6257,2023-04-11T13:40:00.000Z,2023-04-11T10:55:00.000Z,4,balenciaga,1.0
6258,2023-04-11T16:00:00.000Z,2023-04-11T13:40:00.000Z,4,balenciaga,1.0
6259,2023-04-11T18:51:00.000Z,2023-04-11T16:00:00.000Z,5,balenciaga,1.0
6260,2023-04-11T22:11:00.000Z,2023-04-11T18:51:00.000Z,2,balenciaga,1.0


In the next approach, the code iterates over each row to find tweets’ authors, their creation time, their text, the language of texts, their retweet counts, their reply counts, and their locations.
Endpoint has a rate limit of 300 requests/15 minutes, then up to 300 requests over any 15-minute interval are allowed.

In [139]:
MY_BEARER_TOKEN = bt
import time

def extract_geo(s):
    try:
        a = s['place_id']
        return a
    except:
        return None

# df_arr_all = []
counter = 0

for i in tqdm(range(0, ch.shape[0])):
    
    print(ch['start'][i], ch['end'][i])
    
    try:
        client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

        tweets = client.search_all_tweets(query=ch['search_query'][i],
                                         start_time=ch['start'][i],
                                         end_time=ch['end'][i],
                                         tweet_fields = ["created_at", "text", "source", 'author_id', 'public_metrics',
                                                        'geo', 'entities', 'lang'],
                                         user_fields = ["name", "username", "location", "verified", "description"],
                                         place_fields = ['full_name', 'id', 'country', 'country_code', 'geo', 'name',
                                                        'place_type'],
                                         max_results = 500,
                                         expansions=['author_id', 'geo.place_id']
                                         )
    except:
        client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

        tweets = client.search_all_tweets(query=ch['search_query'][i],
                                         start_time=ch['start'][i],
                                         end_time=ch['end'][i],
                                         tweet_fields = ["created_at", "text", "source", 'author_id', 'public_metrics',
                                                        'geo', 'entities', 'lang'],
                                         user_fields = ["name", "username", "location", "verified", "description"],
                                         place_fields = ['full_name', 'id', 'country', 'country_code', 'geo', 'name',
                                                        'place_type'],
                                         max_results = 500,
                                         expansions=['author_id', 'geo.place_id']
                                         )
    counter += 1
    time.sleep(1)
    df_arr = []
    if counter == 290:
        time.sleep(900)
        counter = 0

    if tweets.meta['result_count'] > 0:

        print(f"Was found {tweets.meta['result_count']} tweets")

        for el in tweets.data:
            df_arr.append([el.id, el.text, el.author_id, extract_geo(el.geo), el.created_at, 
                           el.lang, el.public_metrics.get('retweet_count', 0), el.public_metrics.get('reply_count', 0),
                           el.public_metrics.get('like_count', 0), el.public_metrics.get('quote_count', 0)])

        df = pd.DataFrame(df_arr)

        df.columns = ['tweet_id', 'text', 'author_id', 'geo', 'created_at', 'lang', 'retweet_count', 'reply_count',
                     'like_count', 'quote_count']

        df['author_location'] = '-1'

        df['country'] = 'No Country'

        df['query'] = ch['search_query'][i]

        df_arr_all.append(df)

  0%|          | 0/140 [00:00<?, ?it/s]

2023-03-26T11:58:00.000Z 2023-03-26T15:19:00.000Z
Было найдено еще 435 твитов
2023-03-26T15:19:00.000Z 2023-03-26T17:59:00.000Z
Было найдено еще 434 твитов
2023-03-26T17:59:00.000Z 2023-03-26T20:05:00.000Z
Было найдено еще 427 твитов
2023-03-26T20:05:00.000Z 2023-03-26T22:10:00.000Z
Было найдено еще 413 твитов
2023-03-26T22:10:00.000Z 2023-03-27T00:02:00.000Z
Было найдено еще 418 твитов
2023-03-27T00:02:00.000Z 2023-03-27T01:49:00.000Z
Было найдено еще 428 твитов
2023-03-27T01:49:00.000Z 2023-03-27T03:34:00.000Z
Было найдено еще 430 твитов
2023-03-27T03:34:00.000Z 2023-03-27T05:35:00.000Z
Было найдено еще 429 твитов
2023-03-27T05:35:00.000Z 2023-03-27T07:53:00.000Z
Было найдено еще 438 твитов
2023-03-27T07:53:00.000Z 2023-03-27T09:49:00.000Z
Было найдено еще 436 твитов
2023-03-27T09:49:00.000Z 2023-03-27T11:29:00.000Z
Было найдено еще 438 твитов
2023-03-27T11:29:00.000Z 2023-03-27T13:11:00.000Z
Было найдено еще 434 твитов
2023-03-27T13:11:00.000Z 2023-03-27T15:03:00.000Z
Было найдено е

Было найдено еще 437 твитов
2023-04-07T15:52:00.000Z 2023-04-07T17:47:00.000Z
Было найдено еще 435 твитов
2023-04-07T17:47:00.000Z 2023-04-07T19:45:00.000Z
Было найдено еще 439 твитов
2023-04-07T19:45:00.000Z 2023-04-07T21:30:00.000Z
Было найдено еще 433 твитов
2023-04-07T21:30:00.000Z 2023-04-07T23:14:00.000Z
Было найдено еще 437 твитов
2023-04-07T23:14:00.000Z 2023-04-08T01:40:00.000Z
Было найдено еще 437 твитов
2023-04-08T01:40:00.000Z 2023-04-08T05:08:00.000Z
Было найдено еще 437 твитов
2023-04-08T05:08:00.000Z 2023-04-08T07:56:00.000Z
Было найдено еще 442 твитов
2023-04-08T07:56:00.000Z 2023-04-08T11:06:00.000Z
Было найдено еще 436 твитов
2023-04-08T11:06:00.000Z 2023-04-08T14:24:00.000Z
Было найдено еще 435 твитов
2023-04-08T14:24:00.000Z 2023-04-08T17:23:00.000Z
Было найдено еще 444 твитов
2023-04-08T17:23:00.000Z 2023-04-08T20:19:00.000Z
Было найдено еще 444 твитов
2023-04-08T20:19:00.000Z 2023-04-08T22:56:00.000Z
Было найдено еще 435 твитов
2023-04-08T22:56:00.000Z 2023-04-09T

In [142]:
### saving tweets 

pd.concat(df_arr_all, axis=0).to_pickle('balenciaga2.pkl')
dff = pd.concat(df_arr_all, axis=0)
dff.shape

### all in all 2284038 tweets 

(2284038, 13)

Getting user data from batches for 100 users 

In [146]:
users = dff['author_id'].unique()
df = pd.DataFrame({'author_id':users})

In [ ]:
import time
from tqdm.auto import tqdm
import requests

headers = {
    'Authorization': f"Bearer {bt}",
}

# profile_arr = []
counter = 295

for i in tqdm(range(0, df.shape[0] // 100 + 1)):
    if counter == 295:
        time.sleep(900)
        counter = 0
        
    author_ids = ','.join([str(i) for i in df.iloc[i * 100:(i+1)*100, :]['author_id'].tolist()])
    s = f"https://api.twitter.com/2/users?ids={author_ids}&user.fields=location,name"
    response = requests.get(s, headers=headers)
    r = json.loads(response.text)['data']
    profile_arr.append(pd.DataFrame(r))
    counter += 1
    

In [161]:
df = pd.concat(profile_arr, axis = 0).reset_index(drop=True)
df.to_pickle('users_balenciaga.pkl')
df.head()

,username,location,id,name,withheld
0,nickithetaurus,"Las Vegas, NV",1100213555767447556,Thow’d Off,NaN
1,WindYCitYFreScO,NaN,186351277,Sir E G,NaN
2,JoergBredemeier,"Henstedt-Ulzburg, Deutschland",1522475722987786241,Jörg-H. Bredemeier,NaN
3,lemondreamsicle,"NYC, Toronto or Lagos",4863079029,Creamsicle,NaN
4,MaybeBabii,pandora,3284443339,🏹Cooking 𝕄𝔹🏹,NaN
